In [1]:
from abc import ABC, abstractmethod
import re
from sacremoses import MosesTokenizer
import Levenshtein
import spacy
import nltk
import pickle
import urllib
import os
import tarfile
import zipfile
import seaborn as sns
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import numpy as np
# import paths
from string import punctuation
import matplotlib.pyplot as plt
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
from nltk import word_tokenize
RESOURCES_DIR = Path('../resources')
DATASETS_PATH = RESOURCES_DIR / "datasets"
WORD_EMBEDDINGS_NAME ='conllmodel'
DUMPS_DIR = RESOURCES_DIR / "DUMPS"

import gensim
stopwords = set(stopwords.words("dutch"))

In [2]:
def ControlDivisionByZero(numerator, denominator):
    return numerator / denominator if denominator != 0 else 0


class FeatureAbstract(ABC):

    @abstractmethod
    def get_ratio(self, kwargs):
        pass

    @abstractmethod
    def calculate_ratio(self, simple_text, original_text):
        pass


class Feature(FeatureAbstract):

    def __init__(self, split, target_ratio):
        self.split = split
        self.target_ratio = target_ratio

    def get_ratio(self, kwargs):
        if not 'original_text_preprocessed' in kwargs:
            kwargs['original_text_preprocessed'] = ""

        if self.split == "train":
            simple_text = kwargs.get('simple_text')
            original_text = kwargs.get('original_text')
            result_ratio = self.calculate_ratio(simple_text, original_text)

        elif self.split == "valid" or self.split == "test":
            result_ratio = self.target_ratio
        else:
            raise ValueError("stage value not supported")
        kwargs['original_text_preprocessed'] += f'{self.name}_{result_ratio} '
        return kwargs

    @property
    def name(self):
        class_name = self.__class__.__name__
        name = ""
        for word in re.findall('[A-Z][^A-Z]*', class_name):
            if word: name += word[0]
        if not name: name = class_name
        return name


class WordLengthRatio(Feature):

    def __init__(self, stage, target_ratio):
        super().__init__(stage, target_ratio)
        if stage == "train":
            # THIS IS A WORD TOKENIZER, we need one for dutch
            # nl_core_news_sm spacy  spacy.nl_core_news_sm 
            # nltk.word_tokenize
            self.tokenizer =  MosesTokenizer(lang='nl') #  nltk.word_tokenize(language='dutch')  # Moses Tokenizer for Dutch language

    def calculate_ratio(self, simple_text, original_text):
        return round(ControlDivisionByZero(
            len(self.tokenizer.tokenize(simple_text)),
            len(self.tokenizer.tokenize(original_text))), 2)


class CharLengthRatio(Feature):

    def __init__(self, stage, target_ratio):
        super().__init__(stage, target_ratio)

    def calculate_ratio(self, simple_text, original_text):
        return round(ControlDivisionByZero(len(simple_text),
            len(original_text)), 2)


class LevenshteinRatio(Feature):

    def __init__(self, stage, target_ratio):
        super().__init__(stage, target_ratio)

    def calculate_ratio(self, simple_text, original_text):
        simple_text = word_tokenize(simple_text,language='dutch')
        original_text = word_tokenize(original_text,language='dutch')
        return round(Levenshtein.seqratio(original_text,
                                       simple_text), 2)


class DependencyTreeDepthRatio(Feature):

    def __init__(self, stage, target_ratio):
        super().__init__(stage, target_ratio)
        if stage == "train":
            self.nlp = self.get_spacy_model()

    def get_spacy_model(self):

        model = 'nl_core_news_sm'  # from spacy, Dutch pipeline optimized for CPU. Components: tok2vec, morphologizer, tagger, parser, lemmatizer (trainable_lemmatizer), senter, ner.
        if not spacy.util.is_package(model):
            spacy.cli.download(model)
            spacy.cli.link(model, model, force=True, model_path=spacy.util.get_package_path(model))
        return spacy.load(model)

    def calculate_ratio(self, simple_text, original_text):

        result_ratio = round(ControlDivisionByZero(
            self.get_dependency_tree_depth(simple_text),
            self.get_dependency_tree_depth(original_text)), 2)

        return result_ratio

    def get_dependency_tree_depth(self, sentence):

        def get_subtree_depth(node):
            if len(list(node.children)) == 0:
                return 0
            return 1 + max([get_subtree_depth(child) for child in node.children])

        tree_depths = [get_subtree_depth(spacy_sentence.root) for spacy_sentence in self.nlp(sentence).sents]
        if len(tree_depths) == 0:
            return 0
        return max(tree_depths)


class WordRankRatio(Feature):
    # single underscore = internally

    def __init__(self, stage, target_ratio): # constructor of the class 
        super().__init__(stage, target_ratio)
        if stage == "train":
            self.tokenizer = MosesTokenizer(lang='nl')
            self.word2rank = self._get_word2rank()
            print('finished get word2rank')
            # store in file
            # if not present, make file
            # with open ("./resources/DUMPS/word2rank.txt", "a", encoding="utf8") as file: 
            #     file.writelines(self.word2rank)
            #     file.write("\n")
            #     file.close()
            print('length of word2rank', len(self.word2rank))
            self.length_rank = len(self.word2rank) # hier length of the file! 

    def calculate_ratio(self, simple_text, original_text):

        result_ratio = round(min(ControlDivisionByZero(self.get_lexical_complexity_score(simple_text),
                                                       self.get_lexical_complexity_score(original_text)),
                                 2), 2)

        return result_ratio

    def get_lexical_complexity_score(self, sentence, quantile_value=0.75):

        words = self.tokenizer.tokenize(self._remove_stopwords(self._remove_punctuation(sentence)))
        words = [word for word in words if word in self.word2rank]
        if len(words) == 0:
            return np.log(1 + self.length_rank)
        return np.quantile([self._get_rank(word) for word in words], quantile_value)

    def _remove_punctuation(self, text):
        return ' '.join([word for word in self.tokenizer.tokenize(text) if not self._is_punctuation(word)])

    def _remove_stopwords(self, text):
        return ' '.join([w for w in self.tokenizer.tokenize(text) if w.lower() not in stopwords])

    def _is_punctuation(self, word):
        return ''.join([char for char in word if char not in punctuation]) == ''

    def _get_rank(self, word):
        rank = self.word2rank.get(word, self.length_rank)
        return np.log(1 + rank)

    def _get_word2rank(self, vocab_size=np.inf):
        model_filepath = DUMPS_DIR / f"{WORD_EMBEDDINGS_NAME}.pk"
        if model_filepath.exists():
            with open(model_filepath, 'rb') as f:
                model = pickle.load(f)
            return model
        else:
            print("Opening conll model...")
            print("Preprocessing word2rank...")
            DUMPS_DIR.mkdir(parents=True, exist_ok=True)
            WORD_EMBEDDINGS_PATH = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.txt'
            lines_generator = self._yield_lines(WORD_EMBEDDINGS_PATH)
            word2rank = {}
            # next(lines_generator)
            for i, line in enumerate(lines_generator):
                if i >= vocab_size: break
                word = line.split(' ')[0]
                word2rank[word] = i

            pickle.dump(word2rank, open(model_filepath, 'wb'))
            # txt_file = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.txt'
            # zip_file = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.zip'
            # if txt_file.exists(): txt_file.unlink()
            # if zip_file.exists(): zip_file.unlink()
            return word2rank
        
        # else:            
        #     print("Downloading dutch embeddings ...") # pretrained vectors
        #     self._download_twitter_embeddings(model_name='coostco', dest_dir=str(DUMPS_DIR))
        #     print("Preprocessing word2rank...")
        #     DUMPS_DIR.mkdir(parents=True, exist_ok=True)
        #     WORD_EMBEDDINGS_PATH = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.bin'
        #     model = self._load_word_embeddings(WORD_EMBEDDINGS_PATH) # returns index_to_key
        #     # store into file
        #     lines_generator = model # self._yield_lines(model) # (WORD_EMBEDDINGS_PATH)
            
        #     word2rank = {}
        #     # next(lines_generator)
        #     print('vocab_size', vocab_size)
        #     for i, line in enumerate(lines_generator):
        #         if i >= vocab_size: break # its not vocab size any more but  # len(model.key_to_index)
        #         word = line.split(',')[0]
        #         print('word', word)
        #         word2rank[word] = i
        #         print('ranked word?', word2rank[word])
                
        #     pickle.dump(word2rank, open(model_filepath, 'wb'))
        #     txt_file = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.txt'
        #     zip_file = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.zip'
        #     # if txt_file.exists(): txt_file.unlink()
        #     # if zip_file.exists(): zip_file.unlink()
        #     # print(word2rank)
        #     return word2rank
    
    def _load_word_embeddings(self, filepath):
        model = gensim.models.KeyedVectors.load_word2vec_format(filepath, binary=True) # '../resources/DUMPS/model.bin'
        model_indexes = model.index_to_key
        return model_indexes
    
    def _download_twitter_embeddings(self, model_name, dest_dir): # pretrained rankings
        url = ''
        if model_name == 'coosto_model':
            url = 'https://github.com/coosto/dutch-word-embeddings/releases/download/v1.0/model.bin'
        file_path = self._download_url(url, dest_dir)
        out_filepath = Path(file_path)
        out_filepath = out_filepath.parent / f'{out_filepath.stem}.txt'
        # print(out_filepath, out_filepath.exists())
        if not out_filepath.exists():
            print("Extracting: ", Path(file_path).name)
            self._unzip(file_path, dest_dir)

    def _yield_lines(self, filepath):
        filepath = Path(filepath)
        with filepath.open('r', encoding="latin-1") as f:
            for line in f:
                print(line)
                yield line.rstrip()

    def _download_url(self, url, output_path):
        name = url.split('/')[-1]
        file_path = f'{output_path}/{name}'
        if not Path(file_path).exists():
            with tqdm(unit='B', unit_scale=True, leave=True, miniters=1,
                      desc=name) as t:  # all optional kwargs
                urllib.request.urlretrieve(url, filename=file_path, reporthook=self._download_report_hook(t), data=None)
        return file_path

    def _unzip(self, file_path, dest_dir=None):
        if dest_dir is None:
            dest_dir = os.path.dirname(file_path)
        if file_path.endswith('.zip'):
            with zipfile.ZipFile(file_path, "r") as zip_ref:
                zip_ref.extractall(dest_dir)
        elif file_path.endswith("tar.gz") or file_path.endswith("tgz"):
            tar = tarfile.open(file_path, "r:gz")
            tar.extractall(dest_dir)
            tar.close()
        elif file_path.endswith("tar"):
            tar = tarfile.open(file_path, "r:")
            tar.extractall(dest_dir)
            tar.close()

    def _download_report_hook(self, t):
        last_b = [0]

        def inner(b=1, bsize=1, tsize=None):
            if tsize is not None:
                t.total = tsize
            t.update((b - last_b[0]) * bsize)
            last_b[0] = b

        return inner

In [3]:
complex_sentence = "Sommige steden aan de Eyre Highway in de zuidoostelijke hoek van West-Australië, tussen de Zuid-Australische grens bijna tot aan Caiguna, volgen de officiële West-Australische tijd niet."
simple_sentence = "Sommige steden in West-Australië gebruiken geen West-Australische tijd."

In [4]:
wordRank = WordRankRatio("train", 0.8)

finished get word2rank
length of word2rank 2610659


In [5]:
wordRank.get_lexical_complexity_score(complex_sentence)

7.551888702090887

In [6]:
wordRank.get_lexical_complexity_score(simple_sentence)

6.35263850160397

In [7]:
complex_sentence = "Het ruimtevaartuig bestaat uit twee hoofdelementen: de NASA Cassini-orbiter, genoemd naar de Italiaans-Franse astronoom Giovanni Domenico Cassini, en de ESA Huygens-sonde, genoemd naar de Nederlandse astronoom, wiskundige en natuurkundige Christiaan Huygens."
simple_sentence = "Het ruimtevaartuig heeft twee hoofdelementen: de NASA Cassini-orbiter en de ESA Huygens-sonde."

In [8]:
wordRank.get_lexical_complexity_score(complex_sentence)

10.021981446583917

In [9]:
wordRank.get_lexical_complexity_score(simple_sentence)

11.509921149911918

In [10]:
complex_sentence = "Dit was het gebied ten oosten van de monding van de rivier de Vistula, later ook wel 'eigenlijk Pruisen' genoemd."
simple_sentence="Pruisen was eigenlijk de plaats ten oosten van de monding van de rivier de Vistula."

In [11]:
wordRank.get_lexical_complexity_score(complex_sentence)

10.537866200180416

In [12]:
wordRank.get_lexical_complexity_score(simple_sentence)

7.619701861198093

In [13]:
model = gensim.models.KeyedVectors.load_word2vec_format('../resources/DUMPS/conllmodel.bin', binary=True)

In [14]:
model.most_similar("monding")

[('benedenloop', 0.8697854280471802),
 ('bovenloop', 0.8641919493675232),
 ('mondingen', 0.8600053191184998),
 ('stroomopwaarts', 0.851883053779602),
 ('samenvloeiing', 0.8505824208259583),
 ('riviermonding', 0.8490188717842102),
 ('zeearm', 0.8413596153259277),
 ('stroomafwaarts', 0.8371389508247375),
 ('riviermond', 0.830560028553009),
 ('zijrivier', 0.8237558603286743)]

In [15]:
model.most_similar("schiereiland")

[('vasteland', 0.8622255921363831),
 ('eiland', 0.8606902956962585),
 ('reşadiye', 0.8436395525932312),
 ('noordkust', 0.8407957553863525),
 ('alaska-schiereiland', 0.8405270576477051),
 ('noordwestkust', 0.8392339944839478),
 ('zuidpunt', 0.8371938467025757),
 ('zuid-westelijke', 0.8360587954521179),
 ('kuststrook', 0.8331694006919861),
 ('sint-georgekanaal', 0.8328028917312622)]

In [16]:
model.most_similar("chitine") # what to do with that? 

[('mesoglea', 0.8849701881408691),
 ('pectines', 0.8723030686378479),
 ('conchyoline', 0.8718606233596802),
 ('hoornstof', 0.8715182542800903),
 ('kreefachtigen', 0.8686485290527344),
 ('glycoproteïnen', 0.8661853671073914),
 ('vezelachtige', 0.8648804426193237),
 ('verhoornde', 0.8549242615699768),
 ('glycosaminoglycanen', 0.8536439538002014),
 ('kalkzouten', 0.8529971241950989)]

In [17]:
model.most_similar("muiterij")

[('sepoy', 0.8348789215087891),
 ('torpedering', 0.8235708475112915),
 ('muiterijen', 0.8067893385887146),
 ('muiters', 0.8012769222259521),
 ('overmeestering', 0.8012017011642456),
 ('invasiemacht', 0.7950114607810974),
 ('slavenopstand', 0.7922918200492859),
 ('vlooteskader', 0.785224437713623),
 ('kapers', 0.7848634719848633),
 ('java-oorlog', 0.7842100858688354)]

In [18]:
model.most_similar("moslimminderheid")

[('turkomannen', 0.8605157136917114),
 ('mizrachi-joden', 0.8556159734725952),
 ('soennietisch', 0.8548371195793152),
 ('pomaken', 0.8537831902503967),
 ('lipovanen', 0.8486683964729309),
 ('albanisch', 0.8433975577354431),
 ('bevolkingsmeerderheid', 0.8425921201705933),
 ('gagaoezen', 0.8411363959312439),
 ('toevanen', 0.8391970992088318),
 ('wolga-tataren', 0.8386157751083374)]

In [19]:
model.most_similar("vaginale") 

[('vaginaal', 0.8888533711433411),
 ('zaadlozing', 0.8448754549026489),
 ('anale', 0.8444011807441711),
 ('vagina', 0.8265697956085205),
 ('bloedverlies', 0.8188566565513611),
 ('vulva', 0.8148914575576782),
 ('borstvorming', 0.8127890825271606),
 ('candida-infectie', 0.8106294870376587),
 ('dyspareunie', 0.8105209469795227),
 ('perianale', 0.8078418374061584)]

In [20]:
model.most_similar("tandheelkundige")

[('prothetiek', 0.8387660384178162),
 ('orthodontische', 0.8376174569129944),
 ('tandheelkunde', 0.8260310292243958),
 ('tandenbleken', 0.8173831105232239),
 ('kindertandheelkunde', 0.8155919313430786),
 ('orthodontie', 0.8126786947250366),
 ('tandzorg', 0.8124748468399048),
 ('gebitsrenovaties', 0.8113260269165039),
 ('chirurgie', 0.8111916184425354),
 ('angstbegeleiding', 0.8086618185043335)]

In [21]:
model.key_to_index["impasse"]

34928

In [22]:
model.key_to_index["gitaarsnaar"]

753945

In [23]:
model.key_to_index["aardnoten"]

268084

In [24]:
model.most_similar("Warmtekrachtkoppeling")

KeyError: "Key 'Warmtekrachtkoppeling' not present in vocabulary"